In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score
import pandas as pd
import seaborn as sns
from statsmodels import api as sm
from matplotlib import pyplot as plt
from pandas.plotting import register_matplotlib_converters
import os
import numpy as np
import math
import validate
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
import PR_curve

In [7]:
data_dir='fixed_csv_label_ver2'
day=1
original = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
original=original[original['day']<=day]
original

,original,label,time,day
0,53.4,0,15,0
1,52.8,0,15,0
2,52.8,0,15,0
3,53.2,0,15,0
4,53.6,0,15,0
...,...,...,...,...
719465,46.0,0,21,1
719466,46.0,0,21,1
719467,45.9,0,21,1
719468,45.9,0,21,1


In [8]:
df = pd.read_csv( os.path.join(data_dir, 'learning_data_10.csv') )
df=df[df['day']<=day][['original', 'day']]
df

,original,day
0,53.4,0
1,52.8,0
2,52.8,0
3,53.2,0
4,53.6,0
...,...,...
719465,46.0,1
719466,46.0,1
719467,45.9,1
719468,45.9,1


In [9]:
def make_data(x, str,w):
    x=pd.DataFrame(x)
    window=w
    for i in range(1, window):
        column=f'{str}_lag{i}'
        x[column]=x[str].shift(i)
    return x

df_list=[]
for i in set(df['day']):
    lag=[] 
    window=5
    lag.append(make_data(df.loc[df['day']==i, ['original']], 'original', window))
    lag.append(df.loc[df['day']==i, ['day']])
    df_add_lag=pd.concat(lag, axis=1)
    df_list.append(df_add_lag)
df=pd.concat(df_list)
df=df.dropna().reset_index(drop=True)
df['z']=0
df

,original,original_lag1,original_lag2,original_lag3,original_lag4,day,z
0,53.6,53.2,52.8,52.8,53.4,0,0
1,54.0,53.6,53.2,52.8,52.8,0,0
2,54.7,54.0,53.6,53.2,52.8,0,0
3,55.7,54.7,54.0,53.6,53.2,0,0
4,55.5,55.7,54.7,54.0,53.6,0,0
...,...,...,...,...,...,...,...
719457,46.0,46.1,45.9,45.7,45.6,1,0
719458,46.0,46.0,46.1,45.9,45.7,1,0
719459,45.9,46.0,46.0,46.1,45.9,1,0
719460,45.9,45.9,46.0,46.0,46.1,1,0


## 異常ラベル付与

In [5]:
count=1
thr=0.5
while True:
    print(f'-----------------------ラベル付与{count}週目---------------------------')
    for i in set(df['day']):
        print(f'{i}日目')

        test_day=[i]
        train=df[(df['z']==0)&(~df['day'].isin(test_day))][df.columns[:-2]].reset_index(drop=True)
        test=df[(df['day'].isin(test_day))][df.columns[:-2]].reset_index(drop=True)
        num=len(df[(df['day'].isin(test_day))&(df['z']==1)])
        print(f'前の週までで見つかった異常音の個数{num}')

        nk = 10
        neigh = NearestNeighbors(n_neighbors=nk)
        neigh.fit(train)
        d = neigh.kneighbors(test)[0]
        d = np.mean(d, axis=1)
        mx = np.max(d)
        d = d / mx

        test_v=original[original['day'].isin(test_day)][['original', 'label']].reset_index(drop=True)
        test_v['d']=d[0]
        test_v.loc[window-1:, 'd']=d
        d_test=test_v['d'].values

        print(f'閾値{thr}')
        precision, recall=validate.validate(test_v, d_test, thr)
        print(f'適合率：{precision}')
        print(f'再現率：{recall}')

        z=pd.DataFrame(np.where(d>=thr, 1, 0))
        labeling_df=df.loc[(df['day'].isin(test_day))].copy()
        labeling_df.loc[(z[0]==1).values, 'z']=1
        df.loc[(df['day'].isin(test_day))]=labeling_df
        print('-----------------------------------------------')
    print('＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝')
    count+=1
    thr-=0.01

-----------------------ラベル付与1週目---------------------------
0日目
前の週までで見つかった異常音の個数0
閾値0.5
適合率：0.8928571428571429
再現率：0.10256410256410256
-----------------------------------------------
1日目
前の週までで見つかった異常音の個数0
閾値0.5
適合率：1.0
再現率：0.2558139534883721
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数0
閾値0.5
適合率：1.0
再現率：0.021739130434782608
-----------------------------------------------
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
-----------------------ラベル付与2週目---------------------------
0日目
前の週までで見つかった異常音の個数56
閾値0.49
適合率：0.8431372549019608
再現率：0.09401709401709402
-----------------------------------------------
1日目
前の週までで見つかった異常音の個数81
閾値0.49
適合率：1.0
再現率：0.26744186046511625
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数26
閾値0.49
適合率：1.0
再現率：0.021739130434782608
-----------------------------------------------
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
-----------------------ラベル付与3週目---------------------------
0日目
前の週までで見つかった異常音の個数58
閾値0.48
適合率：0.8367346938775511
再現

前の週までで見つかった異常音の個数153
閾値0.32999999999999985
適合率：0.9152542372881356
再現率：0.24786324786324787
-----------------------------------------------
1日目
前の週までで見つかった異常音の個数188
閾値0.32999999999999985
適合率：1.0
再現率：0.4069767441860465
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数40
閾値0.32999999999999985
適合率：1.0
再現率：0.05434782608695652
-----------------------------------------------
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
-----------------------ラベル付与19週目---------------------------
0日目
前の週までで見つかった異常音の個数177
閾値0.31999999999999984
適合率：0.9187817258883249
再現率：0.28205128205128205
-----------------------------------------------
1日目
前の週までで見つかった異常音の個数207
閾値0.31999999999999984
適合率：1.0
再現率：0.45348837209302323
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数43
閾値0.31999999999999984
適合率：1.0
再現率：0.06521739130434782
-----------------------------------------------
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
-----------------------ラベル付与20週目---------------------------
0日目
前の週までで見つかった異常音の

閾値0.1699999999999997
適合率：0.9154185022026432
再現率：0.8953488372093024
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数181
閾値0.1699999999999997
適合率：0.966183574879227
再現率：0.29347826086956524
-----------------------------------------------
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
-----------------------ラベル付与35週目---------------------------
0日目
前の週までで見つかった異常音の個数777
閾値0.1599999999999997
適合率：0.8282485875706215
再現率：0.7863247863247863
-----------------------------------------------
1日目
前の週までで見つかった異常音の個数1135
閾値0.1599999999999997
適合率：0.8993238166791886
再現率：0.8953488372093024
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数207
閾値0.1599999999999997
適合率：0.967741935483871
再現率：0.3695652173913043
-----------------------------------------------
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
-----------------------ラベル付与36週目---------------------------
0日目
前の週までで見つかった異常音の個数885
閾値0.1499999999999997
適合率：0.8128128128128128
再現率：0.8290598290598291
-------------------------------------

閾値0.009999999999999691
適合率：0.10107153917621554
再現率：1.0
-----------------------------------------------
1日目
前の週までで見つかった異常音の個数38791
閾値0.009999999999999691
適合率：0.12854184931634613
再現率：1.0
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数23789
閾値0.009999999999999691
適合率：0.132774104727296
再現率：0.967391304347826
-----------------------------------------------
＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝＝
-----------------------ラベル付与51週目---------------------------
0日目
前の週までで見つかった異常音の個数80725
閾値-3.0878077872387166e-16
適合率：0.03312152042316284
再現率：1.0
-----------------------------------------------
1日目
前の週までで見つかった異常音の個数160827
閾値-3.0878077872387166e-16
適合率：0.08984120910656208
再現率：1.0
-----------------------------------------------
2日目
前の週までで見つかった異常音の個数62467


ValueError: Found array with 0 sample(s) (shape=(0, 5)) while a minimum of 1 is required.